In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, BasicAer, execute
from exp_a4 import A4Factory
from exp_a23 import A23Factory
import numpy as np

In [11]:
# number of qubits n, where size of the matrix is 2**n
n = 3
t = 1
# b entry of the matrices
b_4 = 2
b_2 = 3

#quantum register for the matrix
qr = QuantumRegister(n+1)
#ancilla qubits
qra = QuantumRegister(n-1)
cr = ClassicalRegister(2*n)
qc = QuantumCircuit(qr, qra, cr)

In [12]:
# TODO: substitute H by V
qc.h(qra[0])

# TODO: substitute by suitable exp(iH_jt)

#1 controlled H_1 (here is exp_a4)
A4Factory(n).build_controlled(qc, qr, qr[n], qra, b_4*t)
#0 controlled H_2 (here is exp_a23)
qc.x(qra[0])
A23Factory(n).build_controlled(qc, qr, qr[n], qra, b_2*t)
qc.x(qra[0])

# TODO: substitute H by V^dagger
qc.h(qra[0])

In [ ]:
# TODO: add some measurement